In [6]:

!pip install python-constraint pandas

import pandas as pd
from constraint import Problem, AllDifferentConstraint
from itertools import combinations

# conjunto de datos de transacciones de mercado
data = {
    'TID': [1, 2, 3, 4, 5],
    'Items': [
        ['milk', 'bread', 'butter'],
        ['beer', 'bread'],
        ['milk', 'bread', 'butter'],
        ['beer', 'bread', 'butter'],
        ['milk', 'bread']
    ]
}

df = pd.DataFrame(data)

min_support = 0.6

def find_frequent_itemsets(transactions, min_support):
    items = {item for transaction in transactions for item in transaction}
    problem = Problem()
    problem.addVariables(items, [0, 1])

    def support_constraint(*args):
        selected_items = [item for item, included in zip(items, args) if included]
        count = sum(1 for transaction in transactions if set(selected_items).issubset(set(transaction)))
        return count / len(transactions) >= min_support

    problem.addConstraint(support_constraint, items)

    frequent_itemsets = []
    for solution in problem.getSolutions():
        itemset = [item for item, included in solution.items() if included]
        if itemset:
            frequent_itemsets.append(itemset)

    return frequent_itemsets

transactions = df['Items'].tolist()
frequent_itemsets = find_frequent_itemsets(transactions, min_support)
print("Frequent Itemsets:", frequent_itemsets)

def find_association_rules(frequent_itemsets, transactions, min_confidence):
    rules = []
    for itemset in frequent_itemsets:
        if len(itemset) > 1:
            for i in range(1, len(itemset)):
                for lhs in combinations(itemset, i):
                    rhs = tuple(item for item in itemset if item not in lhs)
                    support_lhs = sum(1 for transaction in transactions if set(lhs).issubset(set(transaction)))
                    support_itemset = sum(1 for transaction in transactions if set(itemset).issubset(set(transaction)))
                    confidence = support_itemset / support_lhs if support_lhs > 0 else 0
                    if confidence >= min_confidence:
                        rules.append((lhs, rhs, confidence))
    return rules

min_confidence = 0.7

association_rules = find_association_rules(frequent_itemsets, transactions, min_confidence)
print("Association Rules:")
for rule in association_rules:
    print(f"{rule[0]} -> {rule[1]} (confidence: {rule[2]})")


Frequent Itemsets: [['bread', 'butter'], ['bread', 'milk'], ['bread'], ['butter'], ['milk']]
Association Rules:
('butter',) -> ('bread',) (confidence: 1.0)
('milk',) -> ('bread',) (confidence: 1.0)
